In [ ]:
from quantinuum_benchmarking.circuit_generation import RandomPauliGadget
from pytket.circuit.display import render_circuit_jupyter
import numpy as np

np.random.seed(2)
n_circuits = 10
seed_list = [np.random.randint(1) for i in range(n_circuits)]

n_qubits = 5

uncompiled_circuit_list = [
    RandomPauliGadget(
        n_qubits=n_qubits,
        depth=n_qubits,
        decompose=False,
        seed=seed,
    ).measure_all() for seed in seed_list
]
render_circuit_jupyter(uncompiled_circuit_list[0])

In [ ]:
from pytket.passes import SequencePass, RemoveRedundancies, DecomposeBoxes
from pauli_gadget_pass import general_pauli_gadget_hi_pass
from hseries_decompositions import hseries_squash, hseries_rebase

hi_circuit_list = [uncompiled_circuit.copy() for uncompiled_circuit in uncompiled_circuit_list]
custom_seq = SequencePass([
    general_pauli_gadget_hi_pass,
    hseries_squash,
    RemoveRedundancies(),
])
for hi_circuit in hi_circuit_list:
    custom_seq.apply(hi_circuit)
render_circuit_jupyter(hi_circuit_list[0])

In [ ]:
std_circuit_list = [uncompiled_circuit.copy() for uncompiled_circuit in uncompiled_circuit_list]
custom_seq = SequencePass([
    DecomposeBoxes(),
    hseries_rebase,
    hseries_squash,
    RemoveRedundancies(),
])
for std_circuit in std_circuit_list:
    custom_seq.apply(std_circuit)
render_circuit_jupyter(std_circuit_list[0])

In [ ]:
from pytket.extensions.myqos import Myqos, MyqosBackend
from pytket.extensions.myqos import QuantinuumConfig, AerConfig

n_shots = 10000

myqos = Myqos()

h1_emul_configuration = QuantinuumConfig(
    device_name="H1-1E",
    user_group='DEFAULT',
    simulator='state-vector',
    noisy_simulation=True,
    no_op=True,
)
h1_ideal_configuration = QuantinuumConfig(
    device_name="H1-1E",
    user_group='DEFAULT',
    simulator='state-vector',
    noisy_simulation=False,
    no_op=True,
)

experiment = myqos.get_experiment_by_name("test_hidden_inverse")

h1_emul_backend = MyqosBackend(h1_emul_configuration, experiment=experiment, remote=True)
h1_ideal_backend = MyqosBackend(h1_ideal_configuration, experiment=experiment, remote=True)

In [ ]:
result_handle_list = h1_emul_backend.process_circuits(hi_circuit_list, n_shots=n_shots)
hi_result_list = h1_emul_backend.get_results(result_handle_list)
hi_counts_list = [hi_result.get_counts() for hi_result in hi_result_list]
hi_counts_list[0]

In [ ]:
result_handle_list = h1_emul_backend.process_circuits(std_circuit_list, n_shots=n_shots, no_opt=True)
std_result_list = h1_emul_backend.get_results(result_handle_list)
std_counts_list = [std_result.get_counts() for std_result in std_result_list]
std_counts_list[0]

In [ ]:
result_handle_list = h1_ideal_backend.process_circuits(std_circuit_list, n_shots=n_shots)
ideal_result_list = h1_ideal_backend.get_results(result_handle_list)
ideal_counts_list = [ideal_result.get_counts() for ideal_result in ideal_result_list]
ideal_counts_list[0]

In [ ]:
from itertools import product

def count_diff(counter_1, counter_2, n):
    return sum(
        abs(counter_1[i] - counter_2[i])
        for i in product(*[[0,1] for _ in range(n)])
    )

for ideal_counts, std_counts, hi_counts in zip(ideal_counts_list, std_counts_list, hi_counts_list):
    print('std_diff', count_diff(ideal_counts, std_counts, n_qubits))
    print('hi_diff', count_diff(ideal_counts, hi_counts, n_qubits))
    if count_diff(ideal_counts, std_counts, n_qubits) < count_diff(ideal_counts, hi_counts, n_qubits):
        print("hidden inverse better")